# Project IF Ensimag - Quantum computing
---

This notebook is the report of a project that deals with the pricing of an option using quantum calculus thanks to the `qiskit`library.


 We first give some explanation about how we usally do to price an option by using the Monte-Carlo method. Then we detail how to build some quantum circuits (encoding of an affine function, comparator, creation of probability distribution) which, when assembled together, form the final circuit that can encode the price of an option. 


**Author** : plp.breton

**Date** : 27 mars 2025


# List of imports

The following list of imports should be sufficient to work on the project.

In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np  
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library.standard_gates import RYGate
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler

from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit_algorithms import IterativeAmplitudeEstimation, EstimationProblem
from itertools import product
from scipy.stats import norm, lognorm


# Standard pricing of a call option

The code below permits to compute the price of a call option using the Black-Scholes closed formula, and then approximating it using a Monte Carlo method.

## Black-Scholes price of the call option

Definition of the function implementing the closed formula for the price of a call option

In [ ]:
def black_scholes_call(S, K, T, r, sigma):
    """
    Calculate the Black-Scholes price for a European call option.
    
    :param S: Current stock price
    :param K: Strike price
    :param T: Maturity (in years)
    :param r: Risk-free interest rate (annual)
    :param sigma: Volatility of the stock (annual)
    :return: Call option price
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

### Characteristics to price the call option

In [ ]:
# Financial characteristics that are also required to compute the probability distribution
sigma = 0.2 # volatility (annualized)
T = 1 # maturity (in years)
r = 0.05 # risk-free rate (annualized)
S0 = 1.5 # initial stock price
K = 1.75 # strike price

Evaluation of the price for the given set of parameters

In [ ]:
call_price = black_scholes_call(S0, K, T, r, sigma)
print(f"Call option price: {call_price}")

## Compounded Black-Scholes price of the call option

In this project, we are actually interested in the compounded value of the call price. Data is stored in dictionaries.

Dictionary for the pricing characteristics

In [ ]:
price_dict = {'S0': S0, 'r': r, 'sigma': sigma, 'T': T}

In [ ]:
def compound_black_scholes_call(price_dict, K):
    """
    Calculate the Black-Scholes price for a European call option.
    
    :param S: Current stock price
    :param K: Strike price
    :param T: Maturity (in years)
    :param r: Risk-free interest rate (annual)
    :param sigma: Volatility of the stock (annual)
    :return: Call option price
    """
    S0 = price_dict['S0']
    T = price_dict['T']
    r = price_dict['r']
    sigma = price_dict['sigma']
    call_price = black_scholes_call(S0, K, T, r, sigma)
    compound = np.exp(r * T)
    return compound * call_price

In [ ]:
compound_call_price = compound_black_scholes_call(price_dict, K)
print(f"Compounded call option price: {compound_call_price}")

## Monte Carlo evaluation of the compounded price of the call option


To simulate asset prices, we use the Black-Scholes process, which represents the asset price at each date t. In particular, at date T, we have the following relation: $S(T) = S(0)\exp{\left[ \left(r - \frac{\sigma²}{2} \right)T + r\epsilon\sqrt{T} \right]}$, where $\epsilon$ follow the law $\mathcal{N}(0,1)$ (we are using the command `random.standard_normal` from **numpy**).

The method consist of simulating many prices with the formula above and take the average as the resulting price.

In [ ]:
def monte_carlo_black_scholes(price_dict, K, num_simulations):
    """
    Monte Carlo simulation for Black-Scholes option pricing.
    
    :param price_dict: Dictionary containing the pricing characteristics
    :param K: Strike price
    :param num_simulations: Number of Monte Carlo simulations
    :return: Option price
    """
    # Retrieve pricing characteristics
    S = price_dict['S0']
    T = price_dict['T']
    r = price_dict['r']
    sigma = price_dict['sigma']
    np.random.seed(1)
    sum = 0
    for i in range(1, num_simulations):
        eps = np.random.standard_normal(1)
        S_T = S*np.exp((r-(sigma**2)/2)*T + sigma*eps*np.sqrt(T))
        p = max(S_T-K, 0)
        sum += p
        
        compound_option_price = sum/num_simulations
    
    return compound_option_price

In [ ]:
monte_carlo_black_scholes(price_dict, K, 50000)

fig, ax = plt.subplots(1,1, figsize=(12, 6))
X = [i for i in range(5000, 100000, 5000)]
y = [monte_carlo_black_scholes(price_dict, K, x) for x in X ]
y_prime = compound_black_scholes_call(price_dict, K)
z = [y_prime - i for i in y]

w = [-0.4*i**(-1/2) for i in X]

ax.plot(X, z)
ax.plot(X, w)

ax.set_xlabel('Number of simulations')
ax.set_ylabel('Option price error')
ax.set_title('Convergence of Monte Carlo simulation to Black-Scholes price')
plt.show()



We can see how the error of the Monte-Carlo method is decreasing by the order $O(M^{-1/2})$ . Thanks to the Quantum Amplitude Estimation algorithm (QAE) we could theorically estimate the undiscounted payoff expectation with an error of order $O(M^{-2/3})$ with $M=2^m$ where $m$ is the number of qubits used for QAE. 

# Storing a linear function in a quantum state

## Presentation
The goal of this part is to construct a circuit that permits to encode an affine function $f: x\mapsto a + bx$ on $n$ qubits, under the constraint that $a$ and $b$ are such that $f(x) \in [0,1]$ for all $x\in \{0, \ldots, 2^n-1\}$.

The encoding is based on the approximation $\sin^2\left(y + \frac{\pi}{4}\right) = y + \frac{1}{2} + O\left(y^3\right)$, where $y$ is replaced by $d\cdot\left(f(x) + \frac{1}{2}\right)$.

## Encoding the affine function

Creation of a method `encode_affine_function`$(n,a,b)$ that encodes the function $f$ defined above into a circuit using rotation gates.

In [ ]:
def encode_affine_function(n, a, b):
    """
    Encode an affine function f(x) = a + b*x into a quantum circuit with result on last qubit.
    
    Args:
        n (int): Number of qubits used to encode the input x
        a (float): Slope of the affine function
        b (float): Intercept of the affine function
    
    Returns:
        QuantumCircuit: Circuit implementing the affine function with result on last qubit
    """
    circuit = QuantumCircuit(n+1)
    
    # Apply constant term to the output qubit (last qubit)
    circuit.ry(2*a, n)
    
    # Apply controlled rotations from input qubits to output qubit
    for i in range(n):
        theta = 2*b*(2**i)
        circuit.cry(theta, i, n)  # Control: input qubit i, Target: output qubit n
    
    return circuit

Let's see an example of the circuit this function returns:

In [ ]:
circuit = encode_affine_function(5, 1, np.pi/2)
circuit.draw('mpl')

Note: We could do the same with all the rotations on the first qubit $q0$ but we have decided to do it on the last qubit as it will make it easier for the final implementation.

## Testing the circuit

Definition of a method `convert_affine`$(a,b,d)$ that outputs values `a_out` and `b_out` for the transformed affine function.

In [ ]:
def convert_affine(a,b,d):
    a_out = d*(a-0.5)+np.pi/4
    b_out = d*b
    return a_out, b_out

The `get_one_prob` method below permits to compute the probability of reading value '1' on qubit $0$ upon a measurement. 

In [ ]:
def get_one_prob(qc, numshots):
    backend = AerSimulator()  
    job = backend.run(qc, shots=numshots)
    result = job.result()
    counts = result.get_counts(qc)
    print(counts)
    
    count_ones = counts.get('1', 0)
    prob = count_ones/numshots
    print(prob)
    return prob


Creation of a method `add_measurement`(`qc`) method that copies circuit `qc`, adds a classical register with a single bit to the new copy and adds a measurement of qubit $0$ with the result stored in the classical bit. The resulting circuit will be used to guarantee that the circuit correctly encodes the affine function.

In [ ]:
def add_measurement_last_bit(qc):
    
    new_reg = ClassicalRegister(1, 'c_reg')
    qc.add_register(new_reg)
    last_qubit = qc.num_qubits - 1
    qc.measure(last_qubit, 0)
    
    return qc

def add_measurement_first_bit(qc):
    
    new_reg = ClassicalRegister(1, 'c_reg')
    qc.add_register(new_reg)
    qc.measure(0, 0)
    
    return qc

### Validation

In [ ]:
# Number of qubits in the circuit
n_test = 3

# Arbitrary value for b
b_test = 1.0/(2**n_test)

# Arbitrary value for a, with the constraint that 0 <= f(x) <= 1 for all x
a_test = 1.0/(2**(n_test+1))

# Arbitrary value for d
d_test = 0.1

In [ ]:
affine_circ = encode_affine_function(n_test, a_test, b_test)

In [ ]:
affine_circ.draw(output='mpl')

In [ ]:
measure_affine = add_measurement_last_bit(affine_circ)

In [ ]:
measure_affine.draw(output='mpl')

Creation of a method `initializer`$(qc,i)$ that takes as an input a quantum circuit $qc$ consisting of $n$ qubits and an integer $0\leq i \leq 2^{n} -1$, and creates a new quantum circuit in which the initial state is $|i\rangle$.

We can do that thanks to the method `QuantumCircuit.inialize()` with the integer $i$ as parameter.

In [ ]:
def initializer(qc, i):
    """
    Initialize the quantum circuit qc in the basis state |i>.
    :param qc: Quantum circuit to initialize
    :param i: Index of the basis state to initialize
    :return: Quantum circuit initialized in the basis state |i>
    """
    n = qc.num_qubits
    if i < 0 or i >= 2**n:
        raise ValueError(f"i must be between 0 and {2**n - 1}")
    
    
    #Note: the Statevector for n qubits is of size 2**n as there are 2**n basis states.
    basis_state = Statevector.from_int(i, 2**(qc.num_qubits-1))
    
    new_qc = QuantumCircuit(qc.num_qubits)
    new_qc.initialize(basis_state, new_qc.qubits[:-1])
    
    new_qc = new_qc.compose(qc)
    
    return new_qc

a_out, b_out = convert_affine(a_test, b_test, d_test)
encoded_qc = encode_affine_function(n_test, a_out, b_out)
initialized_qc = initializer(encoded_qc, 3)
measured_qc = add_measurement_last_bit(initialized_qc)
measured_qc.draw(output='mpl')


We provide below a method `compare_values`$(n,a,b,d,i)$ that compares the expected value 
$$d\cdot\left(f(i)-\frac{1}{2}\right) + \frac{1}{2} = d\cdot\left(a+i\cdot b - \frac{1}{2}\right) + \frac{1}{2}$$
with the probability of measuring value $1$ in the constructed circuit. The method prints the expected value, the probability of measuring the value $1$ and the relative error between both.

This method is in charge of:
- Invoking `convert_affine` to get the appropriate parameters for the encoded affine function
- Creating the initial circuit (`encode_affine_function`)
- Invoking the `initializer` to set the desired initial state
- Adding a classical bit to measure the first qubit (`add_measurement` method) and using the `get_one_prob` method to compute the probability of measuring the value $1$

In [ ]:
def compare_values(n, a, b, d, i):
    a_out, b_out = convert_affine(a, b, d)
    encoded_qc = encode_affine_function(n, a_out, b_out)
    initialized_qc = initializer(encoded_qc,i)
    measured_qc = add_measurement_last_bit(initialized_qc)
    output_prob = get_one_prob(measured_qc, 10000)
    expected_value = d*(a+b*i -0.5) + 0.5
    relative_error =(expected_value-output_prob)/expected_value
    measured_qc.draw(output='mpl')
    print(f'Expected value: {expected_value}, measured output: {output_prob}, relative error: {relative_error:.2%}')

In [ ]:
def compare_all_values(n, a, b, d):
    for i in range(2**n):
        compare_values(n, a, b, d, i)

In [ ]:
compare_all_values(n_test, a_test, b_test, d_test)

As we can see, the circuit is able to store the result of the affine function in the probability distribution. The relative error between the measurement and the expected value is rather low.

# A quantum circuit for integer comparison

The goal of this part is to construct a circuit of size $n$, parameterized by an integer $0\leq L \leq 2^n-1$, that permits to detect whether a given state $|i\rangle$ is such that $i < L$ or $i\geq L$.

## Creation of the 'or' gate

Definition of a method `or_gate`() that constructs a quantum circuit representing an *or* gate using the method described in the exercise sheet. The resulting circuit can be used as a standard gate thanks to the `to_gate()` method. It will be necessary to invoke the `transpile` method before measurement outcomes can be computed.

In [ ]:
def or_gate():
    # We create an "or gate" with the Toffoli gate:
    qc = QuantumCircuit(3)
    qc.x(0)
    qc.x(1)
    qc.x(2)
    qc.ccx(0,1,2)
    qc.x(0)
    qc.x(1)
    return qc.to_gate(label="OR")

In [ ]:
qc = QuantumCircuit(4)
qc.append(or_gate(), [0,1,3])
qc.draw(output='mpl')

We can verify that the *or* gate has the appropriate behavior by computing the probabilities of measuring a $1$ on qubit $2$ for different initial states.


**Useful methods**: 
- The `initialize` method defined above.
- The `to_gate()` instruction can be used to convert a circuit into a gate.
- The `transpile()` instruction allows the AerSimulator to convert the *or* gate back to known gates.

In [ ]:
qc = QuantumCircuit(3)
qc.initialize('001', qc.qubits)
qc.compose(or_gate(), qubits=[2,1,0], inplace=True)
backend = AerSimulator()
transpiled_qc = transpile(qc,backend)
measure = add_measurement_first_bit(transpiled_qc)
prob_measure = get_one_prob(measure, 10000)
measure.draw(output='mpl')
print(f'Probability of measuring 1: {prob_measure}')



## Construction of the comparator circuit

Definition of a `make_comparator`$(n,K)$ method that creates a circuit $q$ of size $2n$ such that, when state $|0^n\rangle\otimes |i\rangle $ is fed as an input, we obtain a state $|x_{2n-1}\cdots x_0\rangle$ as an output with $|x_0\rangle = |0\rangle$ exactly when $i < K$. 


We have seen in the exercise sheet how to constuct an operator that can do exactly the purpose we want. It can be built by dtermining the $\textit{two's complement}$ of $K$, which is $K^{[n]}= \sum_{i=0}^{n-1} ki2^i$ and by constructing operators (either $\textit{OR[]}$ or $\textit{ccNOT[]}$ ) based on the value of $ki$.

In [ ]:
def two_complement(K_bits):

    """
    Compute the two's complement of a binary number K.
    :param K_bits: List of bits representing the binary number K
    :return: List of bits representing the two's complement of K
    """

    complement_K_bits = [1-b for b in K_bits]

    carry = 1
    for i in range(len(complement_K_bits)-1, -1, -1):
        if complement_K_bits[i] == 0 and carry == 1:
            complement_K_bits[i] = 1
            carry = 0
            break
        elif complement_K_bits[i] == 1 and carry == 1:
            complement_K_bits[i] = 0
    return complement_K_bits


def make_comparator(n, L):
    """
    Create the quantum circuit for a comparator that compares an integer i with an integer K.
    For an input |0^n⟩|i⟩, the circuit produces the state |x_{2n-1}...x_0⟩ with x_0 = 0 if i < K (else 1).
    
    Args:
        n: Number of qubits used to encode the integer i
        K: The integer to compare against
    """
    input_reg = QuantumRegister(n, 'x')   
    zero_reg = QuantumRegister(n, '0')    
    
    qc = QuantumCircuit(zero_reg, input_reg)

    # We determine the ki values by computing the two's complement of K
    K_bin = format(L, f'0{n}b')
    K_bits = [0 if b=='0' else 1 for b in K_bin]


    complement_K_bits = two_complement(K_bits)    


    if complement_K_bits[0] == 0: #We apply CNOT[n-1,n] 
        qc.cx(n-1,n)
    

    for j in range(1,n):
        if complement_K_bits[j] == 1: # if kj == 1
            qc.append(or_gate(), [n+j, n-j, n-j-1])
        else:
            qc.ccx(n+j, n-j, n-j-1)
    
    return qc

An example when $n=3$ and $K=5$

In [ ]:
cmp_qc = make_comparator(3, 5)

In [ ]:
cmp_qc.draw(output='mpl')

Definition of a `check_comparison`$(n,L, nb)$ method that invokes the `make_comparator` method, iterates over $0\leq i\leq 2^n-1$ to initialize the circuit with state $|i\rangle \otimes |0^n\rangle$ and verifies that the measurement of the qubit at index $0$ returns the appropriate value for each value of $i$. Parameter $nb$ represents the number of shots used by the sampler to compute measurement outcome statistics.

In [ ]:
def check_comparison(n, L, numshots):
    comp_qc = make_comparator(n, L)
    simulator = AerSimulator(method='statevector')
    
    for i in range(2**n):

        input_reg = QuantumRegister(n, 'x')   
        zero_reg = QuantumRegister(n, '0')  
        qci = QuantumCircuit(zero_reg, input_reg)
        init = [0 for _ in range(2**n)]
        init[i] = 1
        qci.initialize(init, input_reg)
        res_qc = qci.compose(comp_qc)
        meas_qc = add_measurement_first_bit(res_qc)
        
        qc = transpile(meas_qc, simulator)
        
        job = simulator.run(qc, shots=numshots)
        result = job.result()
        counts = result.get_counts(qc)
        
        print(f'Test if {i} >= {L}, measured output: {counts}')
    return qc , comp_qc

In [ ]:
qc , comp_qc = check_comparison(4, 10, 1024)
# comp_qc.draw(output='mpl')
qc.draw(output='mpl')

# Storing a probability distribution in a quantum state

The goal of this part is to design a circuit that permits to store a probability distribution in a quantum state.

## Case where $n=2$

We begin with the simple case where $n=2$. The cell below constructs a circuit permitting to store the probability distribution in a quantum state (with a single qubit). 

In [ ]:
def make_basic_qc(p):
    qreg = QuantumRegister(1)
    qc =  QuantumCircuit(qreg)
    theta = 2*math.acos(math.sqrt(p))
    qc.ry(theta, 0)
    return qc 

In [ ]:
make_basic_qc(0.15).draw(output='mpl')

In [ ]:
def get_state_probabilities(qc):
  n = qc.num_qubits
  statevector = Statevector(qc)
  probabilities = {}
  for i in range(2**n):
    state = bin(i)[2:].zfill(n)
    probabilities[state] = abs(statevector[i])**2  
  return probabilities

In [ ]:
get_state_probabilities(make_basic_qc(0.15))

The probability of getting the outcome $0$ is close to $p$, likewise for $1$ and $1-p$.

## Circuit for an arbitrary value of $n$.

### Generation of the final prices grid

Definition of a method `make_final_price_grid`$(n, \mathrm{price\_dict}, m)$ that generates a regular grid with size $2^n$ of final prices ranging between $\max\left(0, \overline{S} - m\cdot \sigma\right)$ and $\overline{S} + m\cdot \sigma$. This method is based on the `linspace` method from **numpy**.

In [ ]:
def make_final_price_grid(n, price_dict,m):

    S_mean = price_dict['S0']*np.exp(price_dict['r']*price_dict['T'])
    sigma = price_dict['sigma']
    max = np.maximum(0, S_mean - m*sigma)  #corresponds to sm
    return np.linspace(max, S_mean + m*sigma, 2**n)


Definition of a method `align`$(n, K, \mathrm{price\_dict}, m)$ that returns the value $L$ such that $s_L$ is the projection of $K$ on the final price grid. 

In [ ]:
def align(n,K,price_dict,m):
    grid = make_final_price_grid(n, price_dict, m)
    L = 0
    for i in range(2**n):
        if grid[i] >= K:
            L = i
            break
    return (L, grid[L])
    


Test of the discretization error by comparing the outputs of `compound_black_scholes_call` on an arbitrary strike and on its projection on the final price grid.

In [ ]:
init_cpb = compound_black_scholes_call(price_dict, K)
disc_cpb = compound_black_scholes_call(price_dict, align(11,K, price_dict, 3)[1])

print(align(5,K, price_dict, 3))



In [ ]:
print(f'init: {init_cpb}, disc: {disc_cpb}, error: {(init_cpb-disc_cpb)/init_cpb:.2%}')

### Generation of the probability values

Definition of a method `make_probability_distribution` that returns a *last prices* grid and the corresponding value of each $p_i$ for $0 \leq i < 2^n$. The arguments of this method are:
- $n$: the number of qubits under consideration
- $\mathrm{price\_dict}$: the dictionary with the mathematical characteristics of the underlying
- $m$: the number of standard deviations to consider for the minimum and maximum bounds of the grid

***Useful methods:*** 
$S_T$ is of the form $S_0e^{(r-\tfrac{\sigma^2}{2})T}\cdot Z$, where $Z \sim \mathrm{lognorm}(\sigma\sqrt{T})$. The lognormal distribution is available thanks to the `lognorm` method from `scipy.stats`. The `scale` parameter is to be set to $1$.

The probability values can be computed using the `cdf` method available on the lognormal distribution.

In [ ]:
def make_probability_distribution(n, pr_dict, num_stddev):
    """
    Generate the probability distribution for the final stock prices.
    
    :param n: Number of qubits - correspond to 2**n final prices
    :param num_stddev: Number of standard deviations to consider
    :param pr_dict: Dictionary with the princing characteristics
    :return: Probability distribution
    """
    # Generate the log-normal distribution
    S0 = pr_dict['S0']
    r = pr_dict['r']
    T = pr_dict['T']
    sigma = pr_dict['sigma']
    S_mean = S0*np.exp(r*T)
    
    lognorm_dist = lognorm(sigma*np.sqrt(T), scale=1)
    
    final_prices = make_final_price_grid(n, pr_dict, num_stddev)


    sm = final_prices[0]  #max(0, S_mean - num_stddev*sigma)
    sM = S_mean + num_stddev*sigma

    constant= S0*np.exp(r*T - sigma**2*T/2) 
    
    
    # Additional point for the construction of the probability grid

    S_2n = sM + (sM - sm) / (2**n - 1)
    
    # Extension of the final prices grid

    final_prices = np.append(final_prices, S_2n)
    
    # Computation of the quantiles


    first_quantile = lognorm_dist.cdf(sm/constant)
    last_quantile = lognorm_dist.cdf(S_2n/constant)

    normalization_factor = last_quantile - first_quantile

    prob_distribs = []
    for i in range(len(final_prices)-1):
        x1 = final_prices[i]/constant
        x2 = final_prices[i+1]/constant
        quantile1 = lognorm_dist.cdf(x1)
        quantile2 = lognorm_dist.cdf(x2)
        prob_value = quantile2 - quantile1

        prob_value = prob_value / normalization_factor
        prob_distribs.append(prob_value)
    

    prob_distribs.append(prob_distribs[-1]) # It allows to have the same dimension for the final prices and the probability distribution
    return final_prices, prob_distribs


## Validation of the probability distribution, comparison of the Black-Scholes approximation

In [ ]:
# Arbitrary value for n, will afterward be the size of the quantum circuit
n_prob = 13
n_stddev_prob = 4

Verification that the generated distribution sums to $1$.

In [ ]:
final_prices, prob_distribs = make_probability_distribution(n_prob, price_dict, n_stddev_prob)
np.sum(prob_distribs)


Plot the generated distribution against the final price grid.

In [ ]:
plt.plot(final_prices, prob_distribs)
plt.xlabel('i')
plt.ylabel('$p_i$')
plt.show()

Compare the compounded Black-Scholes price of an option with the approximation of the expectation 
$$\mathbb{E}\left[(S_T - s_L)_+\right] \approx \sum_{i=0}^{2^n-1}p_i\cdot(s_i - s_L)_+.$$

In [ ]:
def expectation_cmp(n, L, price_dict, n_stddev):
    last_prices, prob_distribs = make_probability_distribution(n, price_dict, n_stddev)
    sL = last_prices[L]
    sum = 0
    for i in range(0, 2**n):        
        sum += (last_prices[i] - sL)*prob_distribs[i]*(i>=L)
    bs = compound_black_scholes_call(price_dict, sL)
    return {'bs': bs, 'sum': sum, 'strike': sL}

In [ ]:
def get_all_expect_comp(n, price_dict, num_stddev, num_points):
    res = []
    for L in range(0, 2**n,int(2**n/num_points)):
        ex_cmp = expectation_cmp(n, L, price_dict, num_stddev)
        print(f'L: {L} (strike {ex_cmp["strike"]})')
        res.append(ex_cmp)
    return res

In [ ]:
res = get_all_expect_comp(n_prob, price_dict,n_stddev_prob, 20)

In [ ]:
resbs = [x['bs'] for x in res]
ressum = [x['sum'] for x in res]
strikes = [x['strike'] for x in res]
xcoord = price_dict['S0']
plt.plot(strikes, resbs, label='Black-Scholes')
plt.plot(strikes, ressum, label='Approx')
plt.axvline(x=xcoord, color='r', linestyle='--', label=f'S0 = {xcoord}')
plt.legend()
plt.show()


In [ ]:
abs_errors = [(x['sum'] - x['bs']) for x in res]

In [ ]:
plt.plot(strikes, abs_errors)
xcoord = price_dict['S0']
plt.axvline(x=xcoord, color='r', linestyle='--', label=f'x = {xcoord}')
plt.legend()
plt.show()


The error is relatively small and indicate a systematic underestimation by the approximation method but it is not constant.

Indeed the error depends on the strike prices. The approximation is better for high strike prices whereas when it is close to the initial stock price $S0$ and for strike prices lesser than $S0$,the approximation perfoms less well.

## Recursive distribution computations

Computation of the values $q_w$ and $\theta_w$. The values of $q_w$ (resp. $\theta_w$) where $w$ is of length $m$ are stored in a dictionnary for which each key is a string of length $m$. The dictionnaries with the values of $q$ (resp. $\theta$) for $1 \leq m \leq n$ are stored in a dictionnary `DictQ` (resp. `DictTheta`).

Definition of a function `generate_binary_strings`$(n)$ that takes a parameter $n$ and outputs a list of all binary strings of length $n$.

Useful method: `product` from the `itertools` package

In [ ]:
DictQ = dict()
i=10
n=4
# print(f'{i:0{n}b}')
def generate_binary_strings(n):
    list_string = []
    for i in range(2**n):
        s = f'{i:0{n}b}'
        s = "".join(reversed(s))
        list_string.append(s)
    return list_string
generate_binary_strings(4)


Note: As we can see the order of the list of bits is not the classic order, but it fits with the circuit creating the probability distribution (see further).

Definition of a method `make_dict_at_n` that creates the dictionary containing the values $q_w$ where $w$ is of length $n$. This method has two parameters:
- the value of $n$
- the probability distribution under consideration

In [ ]:
def make_dict_at_n(n, prob_distribs):
    list_string = generate_binary_strings(n)
    dict = {}
    for i  in range(len(list_string)):
        dict[list_string[i]] = prob_distribs[i]
    return dict

In [ ]:
n_circ = 3
circ_distrib = {
    0: 0.300,  
    1: 0.250, 
    2: 0.200,  
    3: 0.050,  
    4: 0.050,  
    5: 0.050,
    6: 0.050,
    7: 0.050
}

dict_pn = make_dict_at_n(n_circ, circ_distrib)
print(dict_pn)

In [ ]:
dict_pn

Definition of a method `make_all_dictQ` that iterates from $n-1$ to $1$ to generate all the required probability distributions. Note that the set of keys in the generated dictionary is $\{1, \ldots, n\}$. This method has two parameters:
- the value of $n$
- the dictionary containing the probability distribution at level $n$ (generated by `make_dict_at_n`)

In [ ]:
def make_all_dictQ(n, prob_distribs_dict):
    dictQ = {}
    dictQ[n]= list(prob_distribs_dict.values())
    for m in range(n-1, 0, -1):
        new_prob_distribs = []
        old_prob_distribs = dictQ[m+1]
        for j in range(0,2**(m+1),2):
            new_prob_distribs.append(old_prob_distribs[j] + old_prob_distribs[j+1])
        dictQ[m] = new_prob_distribs
    return dictQ

In [ ]:
dict_all_q = make_all_dictQ(n_circ, dict_pn)
print(dict_all_q)

Definition of a method `make_all_dictTheta` that iterates from $0$ to $n-1$ to generate all the required angles. Note that the set of keys in the generated dictionary is $\{0, \ldots, n-1\}$. This method has a single parameter: the dictionary generated by `make_all_dictQ`.

For the sake of convenience, the element at key $0$ in the generated dictionary will be the angle $\theta^{[0]}_\varepsilon$ instead of a dictionary containing this angle (with the empty string as a key).

In [ ]:
def make_all_dictTheta(dictQ):
    dictTheta = {}
    dictTheta[0] = [2*np.acos(np.sqrt(dictQ[1][0]))]
    for i in range(1,len(dictQ)):
        angle_distribs = []
        for j in range(2**i):
            if dictQ[i][j] == 0:
                theta = 0
            else:
                theta = 2*np.acos(np.sqrt(dictQ[i+1][2*j]/dictQ[i][j]))
            angle_distribs.append(theta)
        dictTheta[i] = angle_distribs
    return dictTheta

In [ ]:
dict_all_theta = make_all_dictTheta(dict_all_q)
print(dict_all_theta)

Definition of a method `make_qc_for_dict` that creates the general quantum circuit for the storage of a quantum distribution. This method has a unique parameter: the dictionary of rotation angles (generated by `make_all_dictTheta`).

Some useful concepts and methods from Qiskit:
- `RYGate` for $y$-rotation gates that are potentially controlled
- `control`: the method to create the controlled version of a quantum gate. This method takes as parameters the number of control qubits, and possibly the state of the control qubits that will trigger the gate (`ctrl_state`).

In [ ]:
def make_qc_for_dict(dict_all_theta):
    n = len(dict_all_theta)
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr)

    # First rotation on the first qubit:
    theta = dict_all_theta[0][0]
    qc.ry(theta, 0)

    for i in range(1,n):
        theta_list = list(dict_all_theta[i])
        string_bits = generate_binary_strings(i)

        for j in range(len(theta_list)):
            theta = theta_list[j]
            # state = "".join(reversed(string_bits[j]))
            state = string_bits[j]
            controlled_ry = RYGate(theta).control(i, ctrl_state=state)

            qc.append(controlled_ry, list(range(i+1)))
           

    return qc    

In [ ]:
prob_distrib_qc = make_qc_for_dict(dict_all_theta)

In [ ]:
def get_one_prob_per_state(qc, numshots,state):
    #This function is very similar to the get_one_prob() function, 
    # but it allows to specify the state for which we want to get the probability.
    
    backend = AerSimulator()
    transpile_qc = transpile(qc, backend) 

    job = backend.run(transpile_qc, shots=numshots)
    result = job.result()
    counts = result.get_counts(qc)
    
    count = counts.get(state, 0)
    prob = count/numshots
    return prob

In [ ]:
prob_distrib_qc.draw(output='mpl')


In [ ]:

# get_one_prob_per_state(prob_distrib_qc, 100000, '000')

## Validation

In [ ]:
circ_prob_distrib = get_state_probabilities(prob_distrib_qc)
print(circ_prob_distrib)

In [ ]:
legend=["Target", "Simulation"]
plot_histogram([dict_pn, circ_prob_distrib], legend=legend, title="Distribution comparison", bar_labels=False)

As we can see on this chart, our circuit thrives to encode probabilities with pretty good results.

Global method that constructs a circuit that stores the probability distribution in a quantum state:

In [ ]:
def make_distribution_circuit(n, pricing_dict, num_stddev):
    _ ,prob_distrib = make_probability_distribution(n, pricing_dict, num_stddev)
    dict_pn = make_dict_at_n(n, prob_distrib)
    dict_all_q = make_all_dictQ(n, dict_pn)
    dict_all_theta = make_all_dictTheta(dict_all_q)
    res = make_qc_for_dict(dict_all_theta)
    return res

# Full integration

In [ ]:
circ_dict = {'n': 3, 'd': 0.01, 'num_stddev': 3, 'm': 3}
K = 2

circ = make_distribution_circuit(circ_dict['n'], price_dict, circ_dict['num_stddev'])
circ.draw(output='mpl')

## Creation of the affine functions $f_0$ and $f_1$

Computation of the values $a_i$ and $b_i$ such that $f_i = a_i + b_i\cdot i$.

In [ ]:
def get_coefficients(circ_dict, price_dict,K):
    n = circ_dict['n']
    d = circ_dict['d']
    num_stddev = circ_dict['num_stddev']
    res = {'a0': np.pi/4 - d/2, 'a1': 0, 'b1': 0, 'L':0, 'sM':0, 'sL':0}
    sigma = price_dict['sigma']


    L, sL = align(n, K, price_dict, num_stddev)
    sm = make_final_price_grid(n, price_dict, num_stddev)[0]
    S_mean = price_dict['S0']*np.exp(price_dict['r']*price_dict['T'])
    sM = S_mean + num_stddev*sigma    
    eps = 0.0000001
    res['b1'] = d*(sM-sm)/((sM-sL)*(2**n-1) + eps)
    res['a1'] = -L*d*(sM-sm)/((sM-sL)*(2**n-1) +eps)
    res['L'] = L
    res['sM'] = sM
    res['sL'] = sL


    return res



Comparison of the expectation to estimate with the expectation of its sine approximation.

In [ ]:
def compare_expect(circ_dict, price_dict, K):
    num_stddev = circ_dict['num_stddev']
    n = circ_dict['n']
    d = circ_dict['d']
    _ ,prob_distrib = make_probability_distribution(n, price_dict, num_stddev)

    mean_f = 0
    mean_sin = 0

    coefs = get_coefficients(circ_dict, price_dict, K)
    # print(coefs['sM']) #debug
    # print(coefs['sL'])
    for i in range(2**n):
        f0 = coefs['a0']
        f1 = coefs['a1'] + coefs['b1']*i
        r = 1 if i >= coefs['L'] else 0
        fc = ((f0 + r*f1) - np.pi/4)/d
        fc = fc + 1/2
        mean_f += prob_distrib[i]*fc
        mean_sin += prob_distrib[i]*(np.sin(f0 + r*f1))**2     

    mean_f = d*mean_f + (1-d)/2   

    
    return mean_f, mean_sin

In [ ]:
expect, sin_expect = compare_expect(circ_dict, price_dict, K)

In [ ]:
expect, sin_expect

Definition of methods to construct the circuits for $f_0$ and $f_1$ using the `encode_affine_function` method.

In [ ]:


def make_affine_function_f0_circ(price_dict, circ_dict, K, i):
    a0 , a1, b1, L , sM, sL= get_coefficients(circ_dict, price_dict, K).values()
    encoded_qc = encode_affine_function(n, 0, a0)
    return encoded_qc


def make_affine_function_f1_circ(price_dict, circ_dict, K, i):
    a0 , a1, b1, L ,sM, sL = get_coefficients(circ_dict, price_dict, K).values()
    encoded_qc = encode_affine_function(n, a1, b1)
    return encoded_qc

make_affine_function_f0_circ(price_dict, circ_dict, K, 3).draw(output='mpl')

## Comparison circuit

Construction of the comparison circuit for a given strike price.

In [ ]:
def make_comparison_circuit(circ_dict, price_dict, K):
    n = circ_dict['n']
    L = get_coefficients(circ_dict, price_dict, K)['L']
    comp_qc = make_comparator(n, L)
    return comp_qc

In [ ]:
cmp_qc = make_comparison_circuit(circ_dict, price_dict, K)

In [ ]:
cmp_qc.draw('mpl')

## Integration

Global integration to construct the global circuit encoding the pricing problem in a quantum state.

In [ ]:
sigma_bis = 0.2 # volatility (annualized)
T_bis = 1 # maturity (in years)
r_bis = 0.05 # risk-free rate (annualized)
S0_bis = 2 # initial stock price
K_bis =  2 # strike price

price = {'S0': S0_bis, 'r': r_bis, 'sigma': sigma_bis, 'T': T_bis}
circ = {'n': 3, 'd': 0.01, 'num_stddev': 3, 'm': 3}

expect, sin_expect = compare_expect(circ, price, K_bis)



In [ ]:
#debug

# prob_distrib = make_distribution_circuit(3, price, 3)
# res_qc = make_distribution_circuit(3, price, 3)
# circ_prob_distrib = get_state_probabilities(prob_distrib_qc)
# print(circ_prob_distrib)
# legend=["Target", "Simulation"]
# plot_histogram([dict_pn, circ_prob_distrib], legend=legend, title="Distribution comparison", bar_labels=False)

In [ ]:

def integration(circ_dict, pricing_dict, K):

    #List of the coefficients
    n = circ_dict['n']
    d = circ_dict['d']
    num_stddev = circ_dict['num_stddev']
    m = circ_dict['m']
    L = get_coefficients(circ_dict, pricing_dict, K)['L']
    a0 = np.pi/4 - d/2
    b0 = 0
    a1 = get_coefficients(circ_dict, pricing_dict, K)['a1']
    b1 = get_coefficients(circ_dict, pricing_dict, K)['b1']

    reg_0 = QuantumRegister(1, 'reg_0')
    reg_1 = QuantumRegister(n-1, 'reg_1')
    reg_2 = QuantumRegister(n, 'reg_2')
    reg_3 = QuantumRegister(1, 'reg_3')
    reg_4 = QuantumRegister(m, 'reg_4')

    qc = QuantumCircuit(reg_0, reg_1, reg_2, reg_3, reg_4)
    dist_circuit = make_distribution_circuit(n, pricing_dict, num_stddev)
    qc.compose(dist_circuit, qubits=reg_2, inplace=True)
    comp_circuit = make_comparator(n, L)
    qc.compose(comp_circuit, qubits=list(reg_0) + list(reg_1) + list(reg_2), inplace=True)

    f0_circuit = encode_affine_function(n, a0, b0)
    qc.compose(f0_circuit, qubits= list(reg_2) + list(reg_3), inplace=True)

    
    f1_circuit = encode_affine_function(n, a1, b1)
    controlled_f1 = f1_circuit.to_gate(label='C_f1').control(1)
    qc.append(controlled_f1, [reg_0[0]] + list(reg_2) + list(reg_3))

    return qc


In [ ]:
qc_pricer = integration(circ, price, K_bis)

In [ ]:
qc_pricer.draw('mpl')

In [ ]:
from qiskit_algorithms import AmplitudeEstimation, FasterAmplitudeEstimation


def qae_eval(qc_pricer,m):
    # qc = qc_pricer.copy()
    # aer_simulator = AerSimulator()
    # qc_transpiled = transpile(qc, aer_simulator)
    
    sampler = Sampler()
    
    obj_qubits = qc_pricer.find_bit(qc_pricer.qregs[3][0])
    objective_qubit = [obj_qubits.index]


    problem = EstimationProblem(
        state_preparation=qc_pricer, 
        objective_qubits=objective_qubit
    )
    
    iae = IterativeAmplitudeEstimation(
        epsilon_target = 0.01,  # Target accuracy
        alpha=0.03,  # Confidence level
        sampler=sampler
    )

 
    fae = FasterAmplitudeEstimation(
    delta=0.01,  # target accuracy
    maxiter=3,  # determines the maximal power of the Grover operator
    sampler=sampler,
    )
    fae_result = fae.estimate(problem)
    
    
    result = iae.estimate(problem)
    # result = fae.estimate(problem)
    return result.estimation
    

In [ ]:
res = qae_eval(qc_pricer,4)

In [ ]:
print(f'sin_expect: {sin_expect}, res: {res}')
print(f'expect: {np.sqrt(expect)}, res: {np.sqrt(res)}')


In [ ]:
error = []
K_values = np.linspace(1.6, 2.5, 20) #outside of this range, K is not on the price gri5
for K in K_values:
    res = qae_eval(integration(circ, price, K), 3)
    expect, sin_expect = compare_expect(circ, price, K)
    error.append((res - expect))

plt.figure(figsize=(12, 6))
plt.plot(K_values, error)
plt.xlabel('Strike price')
plt.ylabel('error')
plt.title('Error in the estimation of the expectation')


Aside from the boundary conditions (>= 2.5), we can see at first glance that the difference between the theoretical value and the measured value at the circuit output is quite small. However, after rescaling, this difference remains significant.


We plot below the value obtained for the option price thanks to the quantum circuit and QAE.

In [ ]:
K_values = np.linspace(1.6, 2.5, 20)
quantum_prices = []
bs_prices = []
qae = []
price = {'S0': 2, 'r': 0.05, 'sigma': 0.2, 'T': 1}
circ['n']= 3
d = circ['d']


for K in K_values:
    
    qc_pricer = integration(circ, price, K)
    res = qae_eval(qc_pricer, 4)
    qae.append(np.abs((res - (1-d)/2)/d))

    
    expect, sin_expect = compare_expect(circ, price, K)
    d = circ['d']
    quantum_prices.append(np.abs((sin_expect - (1-d)/2)/d)  )
    
    # Calculate Black-Scholes price
    bs_price = compound_black_scholes_call(price, K)
    bs_prices.append(bs_price)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(K_values, quantum_prices, 'b-', label='Expectation Price')
plt.plot(K_values, bs_prices, 'r--', label='Black-Scholes')
plt.plot(K_values, qae, 'g-', label='QAE')
plt.xlabel('Strike Price (K)')
plt.ylabel('Option Price')
plt.title('Quantum vs Black-Scholes Option Pricing')
plt.legend()
plt.grid(True)
plt.show()

As we can see, we can rescale the result of the expectation of the payoff and the QAE price so that they can fit with the classical value we want to compute. 


We could smooth the curve of the expectation of the payoff by increasing the value of n, but it would be too heavy to execute. The observation we would make is the same as before that the results are better with a higher strike price.


As for the QAE price, we can see that the circuit sometimes tends to underestimate the expectation, and other times to overestimate it.


# Can we modify or improve these results?

- Modifying the parameter n will infuence largely the results as this parameter chooses the number of qubits used to represent the price.
Moreover, affining the discretization would take too much time to run the quantum computing.


- One solution would be to increase the precision of the QAE algorithm. But, for the `IterativeAmplitudeEstimation` function for example, changing the accuracy parameter (`epsilon_target`) doesn't affect the errors between the theorical expectation and our circuit ouptuts. 

- For now we can't explain these systematic errors. Even if we have validated each block of our circuit, it may be possible that there is a design error in the final assembly. The errors can also come from the errors of the encoding of the affine function. Even if they were rather low, they may have a big influence in the final results. 


 One idea to solve these problems would be to measure the output after each block and compare it with the theoretical measurement to expect and locate which part may undermine the expected results.
